In [1]:
!pip install keras
!pip install sklearn
!pip install matplotlib
!pip install -U -q PyDrive

In [2]:
from keras import backend as K
K.clear_session()

Using TensorFlow backend.


In [0]:
from keras import backend as K
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_import = drive.CreateFile({'id':'1p1bsltfTcIrZ_kfE6kwGTPzcdXorHbb2'})
file_import.GetContentFile('colab_setup.py') 
from colab_setup import setup

setup(drive)

In [6]:
!ls

colab_setup.py			 roc_auc_callback.py	X_train.npy
custom_fast_text_embeddings.npy  sample_submission.csv	X_val.npy
datalab				 train_model.py		y_test.npy
fast_text_embeddings.npy	 X_submission.npy	y_train_full.npy
plot_history.py			 X_test.npy		y_train.npy
__pycache__			 X_train_full.npy	y_val.npy


In [0]:
import numpy as np
import os

from plot_history import plot_history
from roc_auc_callback import RocAucCallback
from train_model import train_with_cv, train_with_submitting, evaluate_on_test

from keras.models import Model
from keras.layers import *
from keras.initializers import *
from keras.layers.merge import concatenate

In [0]:
def one_by_one(filtersNumber, inputLayer, dropout = 0.2):
    one_by_one = Conv1D(filtersNumber, 1, activation = 'elu', padding = 'same', kernel_initializer = 'he_uniform')(inputLayer)
    one_by_one = BatchNormalization()(one_by_one)
    one_by_one = SpatialDropout1D(dropout)(one_by_one)
    
    return one_by_one

In [0]:
def highway_conv_1d(inputLayer, filtersNumber, dropout = 0.2, keepInitialBias = -2):
    highwayInput = Input(shape=(400, 300))
    convolution = one_by_one(filtersNumber, highwayInput, dropout)
    
    keep = Conv1D(
        filtersNumber, 1, 
        activation = 'sigmoid',
        padding = 'same', 
        kernel_initializer = 'he_uniform',
        bias_initializer = Constant(keepInitialBias)
    )(highwayInput)
    
    keep = BatchNormalization()(keep)
    forget = Lambda(lambda layer: 1 - layer)(keep)
    
    gate = add(
        [
            multiply([keep, convolution]),
            multiply([forget, highwayInput])
        ]
    )
    
    return Model(inputs=[highwayInput], outputs = [gate])(inputLayer)

In [0]:
def highway_block(inputLayer, filtersNumber, convolutions = 3):
    block = inputLayer
    for i in range(convolutions):
       block = highway_conv_1d(block, filtersNumber)
    return block

In [0]:
def yoon_kim_feature_extractor(filtersNumber, inputLayer):
    block_1_conv_1 = Conv1D(filtersNumber, 1, activation = 'elu', padding = 'same', kernel_initializer = 'he_uniform')(inputLayer)
    block_1_batchnorm1 = BatchNormalization()(block_1_conv_1)
    block_1_max_pool1 = GlobalMaxPooling1D()(block_1_batchnorm1)

    block_1_conv_2 = Conv1D(filtersNumber, 2, activation = 'elu', padding = 'same', kernel_initializer = 'he_uniform')(inputLayer)
    block_1_batchnorm2 = BatchNormalization()(block_1_conv_2)
    block_1_max_pool2 = GlobalMaxPooling1D()(block_1_batchnorm2)

    block_1_conv_3 = Conv1D(filtersNumber, 3, activation = 'elu', padding = 'same', kernel_initializer = 'he_uniform')(inputLayer)
    block_1_batchnorm3 = BatchNormalization()(block_1_conv_3)
    block_1_max_pool3 = GlobalMaxPooling1D()(block_1_batchnorm3)

    block_1_conv_4 = Conv1D(filtersNumber, 5, activation = 'elu', padding = 'same', kernel_initializer = 'he_uniform')(inputLayer)
    block_1_batchnorm4 = BatchNormalization()(block_1_conv_4)
    block_1_max_pool4 = GlobalMaxPooling1D()(block_1_batchnorm4)

    block_1_features = concatenate([
        block_1_max_pool1, 
        block_1_max_pool2, 
        block_1_max_pool3, 
        block_1_max_pool4
    ])
    block_1_features = Dropout(0.2)(block_1_features)
    return block_1_features

In [12]:
maxWords = 30000
maxSequenceLengthInWords = 400
embeddingDimension = 300
filtersNumber = 300

input_layer = Input(shape=(maxSequenceLengthInWords,))

pretrained_embedding_layer = Embedding(
    maxWords, 
    output_dim=embeddingDimension, 
    input_length=maxSequenceLengthInWords,
    weights = [np.load('fast_text_embeddings.npy')],
    trainable = False
)(input_layer)
pretrained_embedding_layer = SpatialDropout1D(0.2)(pretrained_embedding_layer)

custom_embedding_layer = Embedding(
    maxWords, 
    output_dim=embeddingDimension, 
    input_length=maxSequenceLengthInWords,
    weights = [np.load('custom_fast_text_embeddings.npy')],
    trainable = False
)(input_layer)
custom_embedding_layer = SpatialDropout1D(0.2)(custom_embedding_layer)

embedding = Average()([pretrained_embedding_layer, custom_embedding_layer])
embedding_dropout = SpatialDropout1D(0.2)(embedding)

highway = highway_block(embedding_dropout, 300, 24)

features = yoon_kim_feature_extractor(filtersNumber, highway)

dense_1 = Dense(256, activation = 'elu')(features)
dense_1_normalization = BatchNormalization()(dense_1)
dense_1_dropout = Dropout(0.2)(dense_1_normalization)

dense_2 = Dense(256, activation = 'elu')(dense_1_dropout)
dense_2_normalization = BatchNormalization()(dense_2)
dense_2_dropout = Dropout(0.2)(dense_2_normalization)

output_layer = Dense(6, activation='sigmoid')(dense_2_dropout)

model = Model(inputs=[input_layer], outputs=[output_layer])
            
model.summary()

model.compile(
    loss='binary_crossentropy', 
    optimizer='Adam',
    metrics=['accuracy']
)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 400, 300)     9000000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 400, 300)     9000000     input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 400, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
spatial_dr

In [0]:
history = train_with_cv(model, batchSize=32, rocEvery = 2)

Train on 102124 samples, validate on 25532 samples
Epoch 1/32
 21280/102124 [=====>........................] - ETA: 37:50 - loss: 0.2414 - acc: 0.9215

102112/102124 [============================>.] - ETA: 0s - loss: 0.1663 - acc: 0.9542

102124/102124 [==============================] - 3002s 29ms/step - loss: 0.1663 - acc: 0.9542 - val_loss: 0.2458 - val_acc: 0.9637
Epoch 2/32
 10080/102124 [=>............................] - ETA: 40:40 - loss: 0.1432 - acc: 0.9638

102112/102124 [============================>.] - ETA: 0s - loss: 0.1443 - acc: 0.9629

102124/102124 [==============================] - 2945s 29ms/step - loss: 0.1443 - acc: 0.9629 - val_loss: 0.1562 - val_acc: 0.9637
roc-auc: 0.5122 - roc-auc_val: 0.5082                                                                                                    
Epoch 3/32
  8640/102124 [=>............................] - ETA: 41:18 - loss: 0.1437 - acc: 0.9624

...

102112/102124 [============================>.] - ETA: 0s - loss: 0.1399 - acc: 0.9630

102124/102124 [==============================] - 2954s 29ms/step - loss: 0.1399 - acc: 0.9630 - val_loss: 0.1414 - val_acc: 0.9637
Epoch 12/32
 10048/102124 [=>............................] - ETA: 40:35 - loss: 0.1404 - acc: 0.9628

102112/102124 [============================>.] - ETA: 0s - loss: 0.1394 - acc: 0.9630

102124/102124 [==============================] - 2954s 29ms/step - loss: 0.1394 - acc: 0.9630 - val_loss: 0.1401 - val_acc: 0.9637
roc-auc: 0.4969 - roc-auc_val: 0.4919                                                                                                    
Epoch 13/32
  8608/102124 [=>............................] - ETA: 42:54 - loss: 0.1370 - acc: 0.9635

 27936/102124 [=======>......................] - ETA: 33:16 - loss: 0.1413 - acc: 0.9623

In [0]:
plot_history(history)

In [0]:
print(evaluate_on_test(model))